In [71]:
import pandas as pd
import sklearn
import numpy as np

In [72]:
df_train = pd.read_csv("./transformed/train_small.csv")

df_train = df_train.drop('datetime',axis=1)
df_train

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,price_elec,year,month,day
0,11,0,3,0.031,0,512,1620780,45,29.30,2023,2,2
1,2,0,1,16.900,1,103,309369,62,137.72,2021,2,2
2,7,1,1,502.720,1,159,482105,29,99.66,2022,2,2
3,11,0,1,0.000,0,548,1733480,43,80.38,2023,2,2
4,8,0,3,2.825,0,490,1548778,32,139.64,2023,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
999995,5,1,0,0.000,0,191,584162,21,98.76,2022,2,2
999996,11,0,3,83.824,1,5,16439,45,108.28,2021,2,2
999997,4,1,0,591.908,1,42,125679,17,189.93,2021,2,2
999998,0,0,1,0.106,0,476,1504352,0,226.21,2022,2,2


In [73]:
df_gas = pd.read_csv("./transformed/gas_small.csv")
df_gas = df_gas.drop('data_block_id', axis=1)
df_gas

,forecast_date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,price_gas
0,2021-09-01,45.23,46.32,2021-08-31,45.775
1,2021-09-02,45.62,46.29,2021-09-01,45.955
2,2021-09-03,45.85,46.40,2021-09-02,46.125
3,2021-09-04,46.30,46.80,2021-09-03,46.550
4,2021-09-05,46.30,46.58,2021-09-04,46.440
...,...,...,...,...,...
632,2023-05-26,29.10,34.10,2023-05-25,31.600
633,2023-05-27,28.30,34.10,2023-05-26,31.200
634,2023-05-28,28.10,34.10,2023-05-27,31.100
635,2023-05-29,28.16,36.98,2023-05-28,32.570


In [74]:
# Drop all lines where target is null or nan
# A function 'dropna' exists also
df_train = df_train[df_train["target"].isna() == False]


## Train - Test

In [75]:
from sklearn.model_selection import train_test_split

X = df_train.drop('target', axis=1)
y = df_train['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [76]:
X_train

,county,is_business,product_type,is_consumption,data_block_id,row_id,prediction_unit_id,price_elec,year,month,day
430612,11,0,1,0,216,664502,43,177.84,2022,2,2
175532,11,0,1,0,421,1323982,43,147.21,2022,2,2
518796,15,1,3,0,350,1094842,60,630.12,2022,2,2
203788,14,0,1,0,607,1921718,53,92.00,2023,2,2
989917,7,0,1,0,172,524224,25,110.04,2022,2,2
...,...,...,...,...,...,...,...,...,...,...,...
259243,3,1,3,1,572,1811873,14,97.10,2023,2,2
365929,10,0,3,1,378,1185219,39,194.42,2022,2,2
131961,1,0,1,1,73,218643,6,94.53,2021,2,2
671325,5,1,3,0,565,1789542,23,112.25,2023,2,2


In [77]:
y_train.isna().sum()

0

## Modelisation

[All regression models](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model)

Prendre un model : 
- [Un autre model lineaire](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model)
- [Ensembliste](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble)
- [XGBoost](https://xgboost.readthedocs.io/en/stable/python/python_api.html)

In [78]:
from sklearn.dummy import DummyRegressor

dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(X_train, y_train)

DummyRegressor()

In [79]:
import numpy as np
from sklearn.linear_model import LinearRegression
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

reg = LinearRegression().fit(X_train, y_train)
y_train_hat = reg.predict(X_train)


In [80]:
from xgboost import XGBRegressor

# Initialisez le modèle XGBoost avec des paramètres de base
model = XGBRegressor(n_estimators=20, max_depth=8, learning_rate=1, n_jobs = -1)
# Entraînez le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)

# Faites des prédictions sur l'ensemble de test
predictions = model.predict(X_test)

### Metrics and Evaluation

In [81]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

def model_evaluator(model_trained):
  temp_y_train_hat = model_trained.predict(X_train)
  print("MAE : ", mean_absolute_error(y_train, temp_y_train_hat))
  print("MSE : ", mean_squared_error(y_train, temp_y_train_hat))
  print("MAPE : ", mean_absolute_percentage_error(y_train, temp_y_train_hat))

In [82]:
model_evaluator(dummy_regr)

MAE :  366.30942576407324
MSE :  808609.1130568124
MAPE :  2.1375539555512218e+17


In [83]:
model_evaluator(reg)

MAE :  362.15660560965955
MSE :  710481.132959661
MAPE :  1.6322436433686474e+17


In [84]:
model_evaluator(model)

MAE :  110.2701630511642
MSE :  74420.84646146449
MAPE :  2.9963785482376084e+16


In [85]:
def model_evaluator(model_trained):
  temp_y_test_hat = model_trained.predict(X_test)
  print("MAE : ", mean_absolute_error(y_test, temp_y_test_hat))
  print("MSE : ", mean_squared_error(y_test, temp_y_test_hat))
  print("MAPE : ", mean_absolute_percentage_error(y_test, temp_y_test_hat))

In [86]:
model_evaluator(reg)

MAE :  364.8280350233156
MSE :  725676.7680003381
MAPE :  1.6319123686375475e+17


In [87]:
model_evaluator(model)

MAE :  114.91678994204489
MSE :  86192.36259210993
MAPE :  3.0300806501129492e+16


In [88]:
list_scores = []

for i_estimator in range(0,200,4):
    for depth_estimator in range(1,15,2):
        print(i_estimator,depth_estimator)
        model = XGBRegressor(n_estimators=i_estimator,max_depth=8,learning_rate=1,n_jobs=-1)
#fit model
        model.fit(X_train,y_train)
        y_train_hat = model.predict(X_train)
        y_test_hat = model.predict(X_test)
        list_scores.append({
            "n_estimators" : i_estimator,
            "max_depth": depth_estimator,
            "train_mae":mean_absolute_error(y_train,y_train_hat),
            "test_mae": mean_absolute_error(y_test, y_test_hat)
        })



0 1


0 3
0 5
0 7
0 9
0 11
0 13
4 1
4 3
4 5
4 7
4 9
4 11
4 13
8 1
8 3
8 5
8 7
8 9
8 11
8 13
12 1
12 3
12 5
12 7
12 9
12 11
12 13
16 1
16 3
16 5
16 7
16 9
16 11
16 13
20 1
20 3
20 5
20 7
20 9
20 11
20 13
24 1
24 3
24 5
24 7
24 9
24 11
24 13
28 1
28 3
28 5
28 7
28 9
28 11
28 13
32 1
32 3
32 5
32 7
32 9
32 11
32 13
36 1
36 3
36 5
36 7
36 9
36 11
36 13
40 1
40 3
40 5
40 7
40 9
40 11
40 13
44 1
44 3
44 5
44 7
44 9
44 11
44 13
48 1
48 3
48 5
48 7
48 9
48 11
48 13
52 1
52 3
52 5
52 7
52 9
52 11
52 13
56 1
56 3
56 5
56 7
56 9
56 11
56 13
60 1
60 3
60 5
60 7
60 9
60 11
60 13
64 1
64 3
64 5
64 7
64 9
64 11
64 13
68 1
68 3
68 5
68 7
68 9
68 11
68 13
72 1
72 3
72 5
72 7
72 9
72 11
72 13
76 1
76 3
76 5
76 7
76 9
76 11
76 13
80 1
80 3
80 5
80 7
80 9
80 11
80 13
84 1
84 3
84 5
84 7
84 9
84 11
84 13
88 1
88 3
88 5
88 7
88 9
88 11
88 13
92 1
92 3
92 5
92 7
92 9
92 11
92 13
96 1
96 3
96 5
96 7
96 9
96 11
96 13
100 1
100 3
100 5
100 7
100 9
100 11
100 13
104 1
104 3
104 5
104 7
104 9
104 11
104 13
108 1
108 3


In [89]:
scores = pd.DataFrame(list_scores)

scores.sort_values(by='test_mae')

,n_estimators,max_depth,train_mae,test_mae
349,196,13,90.120852,108.592752
343,196,1,90.120852,108.592752
344,196,3,90.120852,108.592752
348,196,11,90.120852,108.592752
346,196,7,90.120852,108.592752
...,...,...,...,...
4,0,9,272.106949,275.835006
3,0,7,272.106949,275.835006
2,0,5,272.106949,275.835006
1,0,3,272.106949,275.835006


In [90]:
scores.sort_values(by='train_mae')

,n_estimators,max_depth,train_mae,test_mae
349,196,13,90.120852,108.592752
343,196,1,90.120852,108.592752
344,196,3,90.120852,108.592752
348,196,11,90.120852,108.592752
346,196,7,90.120852,108.592752
...,...,...,...,...
4,0,9,272.106949,275.835006
3,0,7,272.106949,275.835006
2,0,5,272.106949,275.835006
1,0,3,272.106949,275.835006
